In [1]:
import sys
!{sys.executable} -m pip install pandas==1.5.2

In [4]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize

import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [5]:
client = EntsoePandasClient(api_key="2d80d663-6eb6-447f-b31b-9d5c6b41365a")


https://github.com/EnergieID/entsoe-py/blob/master/entsoe/mappings.py

country codes and examples:
https://thesmartinsights.com/how-to-query-data-from-the-entso-e-transparency-platform-using-python/


In [3]:
start = pd.Timestamp('20221209', tz='Europe/Brussels')
end = pd.Timestamp('20221210', tz='Europe/Brussels')
country_code = 'AT'  # Belgium
country_code_from = 'FR'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = 'A01'

data = {}


eu_list = ['AT', 'BE', 'BG','HR', 'CZ', 'DE_LU', 'DK', 'EE', 'FI', 'FR', 'GR', 'HU', 'IE', 'IT', 'LV',
          'NL']
for country in eu_list:
    try:
        s = client.query_day_ahead_prices(country, start=start,end=end)
        df = s.to_frame()
        df.rename(columns={df.columns[0]: 'price_mwh'},inplace=True)
        mean_price = df.price_mwh.mean()
        print(mean_price, country)
        #df.head(24)
        data[country] = mean_price
    except:
        print("problem in ", country)
    

411.07599999999996 AT
433.72159999999997 BE
336.0964 BG
359.14880000000005 HR
396.90600000000006 CZ
407.0996 DE_LU
problem in  DK
404.0900000000001 EE
403.52240000000006 FI
443.7228 FR
297.4228 GR
402.346 HU
problem in  IE
problem in  IT
404.0900000000001 LV
409.9275999999999 NL


In [4]:
data

{'AT': 411.07599999999996,
 'BE': 433.72159999999997,
 'BG': 336.0964,
 'HR': 359.14880000000005,
 'CZ': 396.90600000000006,
 'DE_LU': 407.0996,
 'EE': 404.0900000000001,
 'FI': 403.52240000000006,
 'FR': 443.7228,
 'GR': 297.4228,
 'HU': 402.346,
 'LV': 404.0900000000001,
 'NL': 409.9275999999999}

In [5]:
s = client.query_day_ahead_prices('IT_NORD', start=start,end=end)
df = s.to_frame()
df.rename(columns={df.columns[0]: 'price_mwh'},inplace=True)
mean_price = df.price_mwh.mean()
print(mean_price)
df.head(24)

419.38159999999993


,price_mwh
2022-12-09 00:00:00+01:00,336.70
2022-12-09 01:00:00+01:00,329.84
2022-12-09 02:00:00+01:00,318.48
2022-12-09 03:00:00+01:00,310.64
2022-12-09 04:00:00+01:00,309.28
2022-12-09 05:00:00+01:00,321.19
2022-12-09 06:00:00+01:00,363.43
2022-12-09 07:00:00+01:00,456.95
2022-12-09 08:00:00+01:00,477.25
2022-12-09 09:00:00+01:00,484.10


In [6]:
start = pd.Timestamp('20221204', tz='Europe/Brussels')
end = pd.Timestamp('20221205', tz='Europe/Brussels')
t =client.query_generation_per_plant(country_code, start=start,end=end, psr_type=None)
t.head()

,Abwinden-Asten,Altenwörth,Aschach,Block 07 Linz,DO3,FHKW Mellach,Freudenau,GDK-Mellach Linie 10,GDK-Mellach Linie 20,Gerlos,...,Melk,Ottensheim-Wilhering,Reisseck 2,Roßhag,Sim1_DT,Sim1_GT41,Sim1_GT42,Sim3_DT,Wallsee-Mitterkirchen,Ybbs-Persenbeug
,Hydro Run-of-river and poundage,Hydro Run-of-river and poundage,Hydro Run-of-river and poundage,Fossil Gas,Fossil Gas,Fossil Gas,Hydro Run-of-river and poundage,Fossil Gas,Fossil Gas,Hydro Water Reservoir,...,Hydro Run-of-river and poundage,Hydro Run-of-river and poundage,Hydro Pumped Storage,Hydro Pumped Storage,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Hydro Run-of-river and poundage,Hydro Run-of-river and poundage
newlevel,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,...,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated
2022-12-04 00:00:00+01:00,86.0,160.0,119.0,107.0,304.0,0.0,91.0,0.0,0.0,1.0,...,109.0,89.0,0.0,0.0,193.0,255.0,255.0,0.0,111.0,109.0
2022-12-04 01:00:00+01:00,84.0,158.0,119.0,100.0,207.0,0.0,90.0,0.0,0.0,0.0,...,107.0,87.0,0.0,0.0,128.0,167.0,167.0,0.0,109.0,109.0
2022-12-04 02:00:00+01:00,84.0,156.0,121.0,100.0,182.0,0.0,91.0,0.0,0.0,0.0,...,107.0,90.0,0.0,0.0,114.0,147.0,147.0,0.0,109.0,108.0
2022-12-04 03:00:00+01:00,84.0,156.0,121.0,104.0,182.0,0.0,91.0,0.0,0.0,0.0,...,108.0,93.0,0.0,0.0,90.0,113.0,113.0,0.0,111.0,107.0
2022-12-04 04:00:00+01:00,86.0,154.0,124.0,105.0,183.0,0.0,89.0,0.0,0.0,0.0,...,109.0,94.0,0.0,0.0,90.0,113.0,113.0,0.0,110.0,108.0


In [7]:
https://globalenergymonitor.org/projects/global-wind-power-tracker/tracker-map/
https://windeurope.org/intelligence-platform/product/european-offshore-wind-farms-map-public/
https://structurae.net/en/structures/towers/wind-turbines/list

SyntaxError: invalid syntax (1382541109.py, line 1)

In [8]:
def get_list_austria():
    """  """
    

    r = requests.get('https://www.thewindpower.net/country_windfarms_en_13_austria.php')
    # print(r.content)

    df = pd.read_html('https://www.thewindpower.net/country_windfarms_en_13_austria.php', header=0, displayed_only=False)[3]
    # df = pd.read_html('https://www.thewindpower.net/country_windfarms_en_13_austria.php', skiprows = 0, header=0, extract_links='all')[3]
    
    df=df.dropna(subset=['Name'])
    return df

df = get_list_austria()
print(df.shape[0])
df.tail(10)


51


,Name,Area,Power (kW),Number of turbines,Hub height (m),Turbine manufacturer,Status,Commissioning date
84,Freudenau,NaN,600,1,NaN,NaN,Operational,NaN
86,FÃ¼rstkogel,NaN,17250,5,NaN,NaN,Operational,NaN
88,Gaberl,NaN,1200,2,NaN,NaN,Operational,NaN
90,Gattendorf Nord,NaN,12000,4,NaN,NaN,Operational,NaN
92,Gattendorf Nord,NaN,21000,NaN,NaN,NaN,Under construction,NaN
94,Gerasdorf-Seyring I,NaN,600,1,NaN,NaN,Operational,NaN
96,Gerasdorf-Seyring II,NaN,660,1,NaN,NaN,Operational,NaN
98,Gerasdorf-Seyring III,NaN,750,1,NaN,NaN,Operational,NaN
100,GfÃ¶hl,NaN,300,1,NaN,NaN,Operational,NaN
102,Get the full list (325 entries),Get the full list (325 entries),Get the full list (325 entries),Get the full list (325 entries),Get the full list (325 entries),Get the full list (325 entries),Get the full list (325 entries),Get the full list (325 entries)


In [9]:
# https://www.igwindkraft.at/maps/current/
r = requests.get('https://www.igwindkraft.at/src_project/external/maps/generated/gmaps_daten.js')
print(r.encoding)
data = str(r.content).split('= ')[1].split("\'")[0]
d = json.loads(data)

utf-8


In [10]:
#d[0]['places']

In [11]:
df2 = pd.DataFrame.from_dict(d[0]['places'])
df2=df2.data.str.split(",")
df2.head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: data, dtype: float64

In [12]:
#for i in d[0]['places']:
#    print(i['data'])
#df2 = json_normalize(d[0]['places']) 
df2 = pd.read_json(d[0]['places'], orient ='data')
df2.head()

ValueError: Invalid file path or buffer object type: <class 'list'>

In [23]:
import plotly.express as px
import plotly.graph_objects as go
# https://github.com/AReburg/Airbnb-Price-Prediction/tree/main/assets
fig = go.Figure(go.Scattermapbox(mode="markers",lat=[48.235437],lon=[16.901598],marker={'size': 20}, hoverinfo='skip', showlegend=False))


#add_trace(fig, parameters[1], symbol="cafe", name="cafe", size=4)

fig.update_layout(mapbox_style="light", mapbox_accesstoken="pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg", mapbox_zoom=6,
                 mapbox_center={"lat": 48.210033, "lon": 16.363449})
fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
              plot_bgcolor='rgba(0,0,0,0)')
fig.show()

### Plot commodities

Documentation: https://www.commodities-api.com/documentation#historicalrates

Get the lastest data with commodieties API.

In [25]:
class ApiError(Exception):
    pass

start_date = "2021-05-01"
end_date = "2022-05-25"

base_currency = 'TGZ22'
symbol = 'EUR'
endpoint = 'latest'
access_API = 'ns8db94qwty8z0d3qtjd831aytoys40jnnz2e814c0hwft056uy0sgfjwyfi'

#urls = f"https://www.commodities-api.com/api/{endpoint}?access_key={access_API}&base={base_currency}&symbols={symbol}"
#resp = requests.get(urls)

#start_date = "2022-11-08"
#url = f"https://www.commodities-api.com/api/open-high-low-close/{start_date}?access_key={access_API}&base=TGZ22&symbols=EUR"
# time series
resp = requests.get(f"https://www.commodities-api.com/api/timeseries?access_key={access_API}&start_date={start_date}&end_date={end_date}&base={base_currency}\
&symbols={symbol}")
if resp.status_code != 200:
    raise ApiError(f"Can't read OLHC values starting from {start_date}")
print(resp.text)
print(url)

{"data":{"success":false,"error":{"code":505,"type":"timeframe_too_long","info":"The specified timeframe is too long, exceeding 365 days."}}}
https://www.commodities-api.com/api/open-high-low-close/2022-11-08?access_key=ns8db94qwty8z0d3qtjd831aytoys40jnnz2e814c0hwft056uy0sgfjwyfi&base=TGZ22&symbols=EUR


#### Long time series
https://tradingeconomics.com/commodity/eu-natural-gas

future prices: https://www.barchart.com/futures/quotes/TG*0/futures-prices


https://www.barchart.com/futures/quotes/TG*1#google_vignette


In [179]:
def get_commodieties(name):
    if name == 'ttf':
        url=f"https://markets.tradingeconomics.com/chart?s=ngeu:com&interval=1d&span=5y&securify=new&url=/commodity/eu-natural-gas&AUTH=S%2F5%2B8%2BV0tSEqVO3f3fc8i7D1ALNK3vv44AAM1sLXABiksGpLT3z6hHgIC4khJSb1&ohlc=1"
    elif name == 'brent':
        url = "https://markets.tradingeconomics.com/chart?s=co1:com&span=5y&securify=new&url=/commodity/eu-natural-gas&AUTH=S%2F5%2B8%2BV0tSEqVO3f3fc8i7D1ALNK3vv44AAM1sLXABiksGpLT3z6hHgIC4khJSb1&ohlc=1"
    elif name == 'usdrub':
        url = "https://markets.tradingeconomics.com/chart?s=usdrub:cur&interval=1d&span=5y&securify=new&url=/russia/currency&AUTH=ZunHTz66JfXqLQTeiv78irx2z508SOi1CsDlRXaVUGUxcEGnev2LDgoaqNeHISd9&ohlc=1"
    elif name == "nbp":
        url = "https://markets.tradingeconomics.com/chart?s=nguk:com&interval=1d&span=5y&securify=new&url=/russia/currency&AUTH=ZunHTz66JfXqLQTeiv78irx2z508SOi1CsDlRXaVUGUxcEGnev2LDgoaqNeHISd9&ohlc=1"
    elif name == "CO2":
        url = "https://markets.tradingeconomics.com/chart?s=eecxm:ind&interval=1d&span=5y&securify=new&url=/commodity/carbon&AUTH=GPVSlEsoJTkYP45bnbNF28FlnikmX%2BwnW7RXI%2FnXTNxH1U1lhv9220niYGHm%2Fnda&ohlc=1"
    resp = requests.get(url)
    if resp.status_code != 200:
        raise ApiError(f"Problem")
    # print(resp.text)
    #t = str(resp.text)
    json_object = json.dumps(resp.text, indent=4)
    with open(f"{name}.json", "w") as outfile:
        outfile.write(resp.text)

#get_commodieties('ttf')
#get_commodieties('brent')
#get_commodieties('usdrub')
#get_commodieties('nbp')
get_commodieties('CO2')

In [113]:
def get_historic_dates(filename="history.json"):
    f = open(f'{filename}')
    data = json.load(f)
    f.close()
    df_history = json_normalize(data)
    df_history['date'] = pd.to_datetime(df_history['date'], infer_datetime_format=True)
    return df_history

In [182]:
def get_value_at_day(df, date):
    val = df[df['date'] == date]['y'].iloc[0]
    return val

def add_annotation2(fig, df):
    df_history = get_historic_dates()
    for index, row in df_history.iterrows():
        if int(row['priority']) < 2:
            y = get_value_at_day(df, row['date'])
            fig.add_annotation(x=row['date'], y=y,
                text=row['text'], showarrow=True, yshift=10, font=dict(family="Courier New, monospace", size=12, color="#636363"),
                align="center", arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363",)
    return fig


def add_annotation(fig, df, date, text):
    date_time_obj = datetime.strptime(date, '%Y-%m-%d')
    y = get_value_at_day(df, date)
    fig.add_annotation(x=date_time_obj, y=y,
        text=text, showarrow=True, yshift=10, font=dict(family="Courier New, monospace", size=12, color="#636363"),
        align="center", arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363",)
    return fig


def load_historic_chart_data(filename):
    f = open(f'{filename}.json')
    data = json.load(f)
    f.close()
    data = data['series'][0]['data']
    #print(data[0])

    df = json_normalize(data)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True) #, format='%Y-%m%d:%H:%M:%S.%f')

    df = df.iloc[-400:-1, 0:-1]
    return df


def candlestick_chart(filename="ttf", y_axis_title="", title=""):
    df = load_historic_chart_data(filename)
    fig = go.Figure(data=[go.Candlestick(x=df["date"], open=df["open"], high=df["high"],
                    low=df["low"], close=df["close"])])
    fig.update_layout(title=title)
        
    #add_annotation(fig, df, '2022-09-26', "Nordstream 2 <br>Explosion")
    #add_annotation(fig, df, '2022-02-24', "Russian invasion <br>of Ukraine")
    add_annotation2(fig, df)
    fig.update(layout_xaxis_rangeslider_visible=False)
    
    
    fig.update_yaxes(tickfont=dict(family='Helvetica', size=12, color='#9c9c9c'), titlefont=dict(size=12), title_font_color='#9c9c9c', title_text=y_axis_title, mirror=True,
    ticks='outside', showline=True, gridwidth=1, gridcolor='#4c4c4c')
    fig.update_xaxes(tickfont=dict(family='Helvetica', size=12, color='#9c9c9c'), titlefont=dict(size=12), title_font_color='#9c9c9c', title_text="", mirror=True,
    ticks='outside', showline=True, gridwidth=1, gridcolor='#4c4c4c')
    fig.update_layout(font=dict(family="Helvetica"), legend_font_size=14)

    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    #fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()


In [185]:
candlestick_chart(filename="CO2", y_axis_title="Euros per tonne of carbon dioxide (CO2e)", title="EU Carbon Permits<br><sub>CO2 EUROPEAN EMISSION ALLOWANCES</sub>")

In [181]:
candlestick_chart(filename="ttf", y_axis_title="spot price in €/MWh", title="Natural Gas EU<br><sub>Dutch TTF</sub>")

In [178]:
candlestick_chart(filename="nbp", y_axis_title="spot price in GBp/Thm", title="Natural Gas UK<br><sub>NBP</sub>")

In [184]:
candlestick_chart(filename="brent", title="Crude Oil<br><sub>dsdfsdf</sub>")

In [168]:
candlestick_chart(filename="usdrub", title="USD/RUB")

In [167]:
candlestick_chart(filename="henry_hub", title="Henry Hub Natural Gas<br> USD per million BTU")

FileNotFoundError: [Errno 2] No such file or directory: 'henry_hub.json'